In [21]:
import pandas as pd
import gc
from sklearn.model_selection import train_test_split




In [22]:
import importlib
import utils
importlib.reload(utils)


print(dir(utils))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'compute_null_values', 'count_nb_lines', 'count_target_value', 'extract_division_names', 'pa', 'pc', 'pq', 'siren_filter', 'var_filter']


In [23]:
#gestion mémoire  (l'équivalent de free en C)

gc.set_threshold(500, 5, 5)
gc.collect()

0

In [24]:
df_avec_details_financiers=pd.read_parquet("datasets_finaux/dataset_avec_details_financiers.parquet")

In [25]:
print(df_avec_details_financiers.shape)

(36388, 68)


In [26]:
df_avec_details_financiers.head(10)

,siren,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,...,Poids_BFR_exploitation_sur_CA_jours_std,Rotation_des_stocks_jours_tcam,Rotation_des_stocks_jours_moyen,Rotation_des_stocks_jours_std,Credit_clients_Jours_tcam,Credit_clients_Jours_moyen,Credit_clients_Jours_std,Credit_fournisseurs_Jours_tcam,Credit_fournisseurs_Jours_moyen,Credit_fournisseurs_Jours_std
0,480086784,-1,0,3,1,2023,1,7022,0,0,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,43.591,NaN
1,514943323,-1,0,3,1,2023,1,7022,-1,0,...,NaN,0.0,22.369,NaN,0.0,0.000,NaN,0.0,33.286,NaN
2,519715593,2,2023,2,1,2023,1,6209,-1,0,...,NaN,0.0,5.602,NaN,0.0,39.735,NaN,0.0,42.464,NaN
3,520267881,-1,0,2,1,2023,0,7022,0,0,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,52.993,NaN
4,522761139,21,2023,5,1,2023,1,7022,-1,0,...,NaN,0.0,0.000,NaN,0.0,78.778,NaN,0.0,21.579,NaN
5,522807981,12,2023,3,1,2023,1,6399,-1,0,...,NaN,0.0,37.858,NaN,0.0,70.764,NaN,0.0,152.765,NaN
6,523617173,1,2023,6,1,2023,1,6201,-1,0,...,NaN,0.0,7.751,NaN,0.0,354.581,NaN,0.0,102.188,NaN
7,524157872,-1,0,2,0,0,0,7022,0,0,...,NaN,0.0,0.000,NaN,0.0,0.000,NaN,0.0,73.204,NaN
8,524607868,-1,0,4,1,2023,1,6201,-1,0,...,NaN,0.0,0.000,NaN,0.0,23.357,NaN,0.0,0.813,NaN
9,525268975,-1,0,4,1,2023,0,7022,-1,0,...,NaN,0.0,104.706,NaN,0.0,3.013,NaN,0.0,70.758,NaN


In [27]:
#SUPPRESSION DES SIRENS
df_avec_details_financiers=df_avec_details_financiers.drop(["siren"],axis=1)

In [28]:
df_avec_details_financiers.columns

Index(['trancheEffectifsUniteLegale', 'anneeEffectifsUniteLegale',
       'nombrePeriodesUniteLegale', 'categorieEntreprise',
       'anneeCategorieEntreprise', 'etatAdministratifUniteLegale',
       'activitePrincipaleUniteLegale', 'economieSocialeSolidaireUniteLegale',
       'societeMissionUniteLegale', 'nbEtablissements',
       'Chiffre_d_affaires_tcam', 'Chiffre_d_affaires_moyen',
       'Chiffre_d_affaires_std', 'Marge_brute_tcam', 'Marge_brute_moyen',
       'Marge_brute_std', 'EBE_tcam', 'EBE_moyen', 'EBE_std', 'EBIT_tcam',
       'EBIT_moyen', 'EBIT_std', 'Resultat_net_tcam', 'Resultat_net_moyen',
       'Resultat_net_std', 'Taux_d_endettement_tcam',
       'Taux_d_endettement_moyen', 'Taux_d_endettement_std',
       'Ratio_de_liquidite_tcam', 'Ratio_de_liquidite_moyen',
       'Ratio_de_liquidite_std', 'Ratio_de_vetuste_tcam',
       'Ratio_de_vetuste_moyen', 'Ratio_de_vetuste_std',
       'Autonomie_financiere_tcam', 'Autonomie_financiere_moyen',
       'Autonomie_financier

In [29]:
df_avec_details_financiers.var()

trancheEffectifsUniteLegale        1.527636e+01
anneeEffectifsUniteLegale          9.359101e+05
nombrePeriodesUniteLegale          1.211760e+00
categorieEntreprise                2.281899e-01
anneeCategorieEntreprise           5.726366e+05
                                       ...     
Credit_clients_Jours_moyen         1.098621e+08
Credit_clients_Jours_std           2.151265e+08
Credit_fournisseurs_Jours_tcam     2.827669e+06
Credit_fournisseurs_Jours_moyen    3.193283e+07
Credit_fournisseurs_Jours_std      7.141303e+07
Length: 67, dtype: float64

In [30]:
proportion_survie_avec_details = (df_avec_details_financiers['etatAdministratifUniteLegale'] == 1).mean() 
print(proportion_survie_avec_details)

0.7539848301637903


In [31]:
def split(dataset,seed=42,yes_validation=False):
    x=dataset.drop(["etatAdministratifUniteLegale"],axis=1)
    y=dataset["etatAdministratifUniteLegale"]
    if(yes_validation==True): #crée un validation set
        x_train, x_temp, y_train, y_temp = train_test_split( x, y, test_size=0.3, random_state=seed,stratify=y ) 
        x_validation, x_test, y_validation, y_test = train_test_split( x_temp, y_temp, test_size=0.5, random_state=seed,stratify=y_temp )
        return x_train,x_test,y_train,y_test,x_validation, y_validation
    else:
        x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, random_state=seed,stratify=y )
        return x_train,x_test,y_train,y_test


In [32]:
x_train,x_test,y_train,y_test=split(df_avec_details_financiers,seed=42) #possibilité de changer de dataset

#Avec validation set
#x_train,x_test,y_train,y_test,x_validation, y_validation=split(df_avec_details_financiers,seed=42,yes_validation=True)


In [33]:
x_test.isna().sum()

trancheEffectifsUniteLegale           0
anneeEffectifsUniteLegale             0
nombrePeriodesUniteLegale             0
categorieEntreprise                   0
anneeCategorieEntreprise              0
                                   ... 
Credit_clients_Jours_moyen         3150
Credit_clients_Jours_std           7685
Credit_fournisseurs_Jours_tcam        0
Credit_fournisseurs_Jours_moyen    1628
Credit_fournisseurs_Jours_std      6715
Length: 66, dtype: int64

In [34]:
x_train.isna().sum()

trancheEffectifsUniteLegale            0
anneeEffectifsUniteLegale              0
nombrePeriodesUniteLegale              0
categorieEntreprise                    0
anneeCategorieEntreprise               0
                                   ...  
Credit_clients_Jours_moyen          7380
Credit_clients_Jours_std           18128
Credit_fournisseurs_Jours_tcam         0
Credit_fournisseurs_Jours_moyen     3811
Credit_fournisseurs_Jours_std      15735
Length: 66, dtype: int64

In [35]:
cols_to_fill = [col for col in x_train.columns 
                if col.endswith(('_moyen', '_std'))]

medians_train = x_train[cols_to_fill].median()



for col in cols_to_fill:
    x_train[col] = x_train[col].fillna(medians_train[col])


for col in cols_to_fill:
    x_test[col] = x_test[col].fillna(medians_train[col]) #pour éviter la triche
 
#Avec validation set (à décommenter si usage de validation set)
# for col in cols_to_fill:
#     x_validation[col] = x_validation[col].fillna(medians_train[col]) #pour éviter la triche
 


In [36]:
x_test.isna().sum().sum()

np.int64(0)

In [37]:

x_train.isna().sum().sum()

np.int64(0)

In [38]:
y_train.isna().sum()

np.int64(0)

In [39]:
y_test.isna().sum()

np.int64(0)

In [40]:
# #LIBERER MEMOIRE

# del df_avec_details_financiers

# gc.collect()